# Natural Language Processing Project

### What Programming Language are People using for their BitCoin Related Repositories on GitHub?
### Give me the README, and I'll Predict the Language. 

-----

# About this Project

-----

### Project Goal: 

The goal is to scrape README pages from BitCoin related repositories on GitHub, so that we can predict the language of that repository. 

### Project Desciption: 

Language Predictor. This is a group project where we will be scraping GitHub repository README files related to BitCoin. Exploring the text in the README will allow us to identify We will then produce a classification model that will be able to predict the programming language of that repository using only the text in the README. 

- List of the languages whose detection is supported: 
    - JavaScript
    - Python
    - Go
    - C++
    - Java
    - TypeScript
    - HTML
    - PHP
    - C#

--------

# Housekeeping

-----

### Data dictionary

A list of the variables in the dataframe and their meaning.

|Column_Name|Description|
|-----------|-----------|
|||
|||
|||
|||
|||

### Imports

Here are the imports needed to run this notebook.

In [13]:
import pandas as pd
import numpy as np

# Scraping
import requests
from bs4 import BeautifulSoup

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import nltk.sentiment

from wordcloud import WordCloud
pd.set_option('display.max_colwidth', -1)

# Regex
import re

# Time
from time import strftime

import unicodedata
import json
from pprint import pprint

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Custom Imports
import acquire 

# Turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

--------

# Let's Get Started...

-----

## Acquire & Prepare

### Steps Taken to Prepare the Data:

## Exploration

### Initial Hypotheses/Questions:

#### Initial Hypothesis: 

#### Initial Questions: 
    1. 
    2. 
    3. 
    4. 

### Explore: 

### Exploration Takeaways:

### Features to Move Forward with:

## Modeling

-----

# Conclusion

----

### Summary

### Next Steps